This Jupyter notebook is a testing ground for the given problem and shows the values each function will return. Note the notebook is modified to not use the dbtools package and also read from the json provided as a file

In [1]:
import json
from flatten_json import flatten
from jsonpath import jsonpath as jp
# Not using the DB class so DB will be a nonetype object
DB =None

def parse_json(json_data):
    """
    This function will parse the given json body in the API call
    It will return a blueprint of the data which will aid in creating and loading tables
    :param json_data: json data passed in the API call
    :return: a dictionary consisting of the following:-
             1. data type of the columns against each table
             2. list of table names
             3. columns available in each table
             4. values of each column in tha API call for each table
             5. relationship of tables with each other

    """
    with open(json_data, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    #data = json.loads(json_data)
    flattened_data = flatten(data, separator='.')
    data_type_data = {}
    tables_list = set()
    table_column ={}
    table_values = {}
    parent_child_tables = {}
    for key, value in flattened_data.items():
        if '.' in key:
            # Finding the list of different tables
            if not key.split('.')[-2].isnumeric():
                tables = key.split('.')[-2]
                is_list = False
            else:
                tables = key.split('.')[-3]
                is_list = True
            tables_list.add(tables)
            # Enlisting columns for each table
            if not table_column.get(tables):
                table_column[tables] = set()
            table_column[tables].add(key.split('.')[-1])
            # Enlisting the values for each column and their data types against each table
            if not table_values.get(tables):
                if not is_list:
                    table_values[tables] = {}
                    data_type_data[tables] = {}
                else:
                    table_values[tables] = [{}]*len(jp(data,'$.{key}'.format(key='.'.join(key.split('.')[0:-3]))))
                    data_type_data[tables] =  [{}]*len(jp(data,'$.{key}'.format(key='.'.join(key.split('.')[0:-3]))))
            if not is_list:
                table_values[tables].update({key.split('.')[-1]: value})
                data_type_data[tables].update({key.split('.')[-1]: type(value)})
            else:
                table_values[tables][int(key.split('.')[-2])].update({key.split('.')[-1]: value})
                data_type_data[tables][int(key.split('.')[-2])].update({key.split('.')[-1]: type(value)})
            # Relation tables have with each other
            if not parent_child_tables.get(tables):
                parent_child_tables[tables] = set()
            try:
                if not is_list:
                    parent_child_tables[tables].add(key.split('.')[-3])
                else:
                    parent_child_tables[tables].add(key.split('.')[-4])
            except:
                pass
    return {
            'data_type_data': data_type_data,
            'tables_list': tables_list,
            'table_column': table_column,
            'table_values': table_values,
            'parent_child_tables': parent_child_tables
            }


def create_load_automated_tables(DB, json_data):
    """
    This function will be used to create automated tables and insert values into them
    :param DB: An object of the DB tools class
    :param json_data: json data passed in the API call
    """
    # Parse the json and get a blueprint
    meta_parse_json = parse_json(json_data)
    # Iterate over the table list from the meta data received and perform creation and insertion funcions
    for table in meta_parse_json['tables_list']:
        table_name = table
        table_columns = ''
        column_names = ''
        # Build create table queries for tables that have dictionary values in the provided json
        if isinstance(meta_parse_json['data_type_data'][table_name], dict):
            for key, value in meta_parse_json['data_type_data'][table_name].items():
                new_column = '"' + key + '"' + ' ' + return_datatype_for_postgres(value) + ',  '
                table_columns += new_column
        # Build create table queries for tables that have list values in the provided json
        elif isinstance(meta_parse_json['data_type_data'][table_name], list):
            for key, value in meta_parse_json['data_type_data'][table_name][0].items():
                new_column = '"' + key + '"' + ' ' + return_datatype_for_postgres(value) + ',  '
                table_columns += new_column
        # create the given table
        create_sql(DB, table_name, table_columns)
        params = {}
        table_columns = ''
        # Build the insert queries for the tables which have dictionary values
        if isinstance(meta_parse_json['table_values'][table_name], dict):
            for key, value in meta_parse_json['table_values'][table_name].items():
                table_columns += '"' + key + '",'
                column_names += '%(' + key + ')s,'
                params[key] = value if value else None
            insert_sql(DB, table_name, table_columns, column_names, params)
        # Build the insert queries for the tables which have an array of values
        elif isinstance(meta_parse_json['table_values'][table_name], list):
            for item in meta_parse_json['table_values'][table_name]:
                for key, value in item.items():
                    table_columns += '"' + key + '",'
                    column_names += '%(' + key + ')s,'
                    params[key] = value if value else None
                insert_sql(DB, table_name, table_columns, column_names, params)


def return_datatype_for_postgres(datatype):
    """
    convert a python data type to one that postgres can understand in the create queries
    :param datatype: python data type
    :return: postgres data type
    """
    data_type = datatype.__name__
    if datatype == 'datetime':
        data_type = 'timestamp'
    elif data_type == 'int':
        data_type = 'bigint'
    elif data_type == 'dict':
        data_type = 'json'
    else:
        data_type = 'str'
    return data_type


def create_sql(DB, table_name, table_columns):
    """
    This will help build the create table sql and execute it
    :param DB: An object of the DB tools class
    :param table_name: name of the table to be created
    :param table_columns: string of table column names along with their data type
    """
    create_sql = '''
                         CREATE TABLE IF NOT EXISTS
                         "{table_name}"
                         ({table_columns}
                         );
                    '''.format(table_name=table_name,
                               table_columns=table_columns,
                               )
    #DB.execute_sql(create_sql)
    print(create_sql)


def insert_sql(DB, table_name, table_columns, column_names, params):
    """

    :param DB: An object of the DB tools class
    :param table_name: name of the table to which values will be inserted
    :param table_columns: name of the columns in the tables
    :param column_names: its a parameterized format of the column names which will help in executing the sql
    :param params: a dictionary consisting of values to be inserted against its column names
    """
    insert_sql = '''
                    INSERT INTO 
                        "{table_name}"
                    ({table_columns})
                    VALUES
                    ({column_names})
                 '''.format(table_name=table_name,
                            table_columns=table_columns,
                            column_names=column_names)
    #DB.execute_sql(insert_sql, params)
    print(insert_sql%params)

In [2]:
meta_parse_json = parse_json('sample.json')

In [3]:
# List of tables in the given Json
meta_parse_json['tables_list']

{'company',
 'contacts',
 'object',
 'payment_methods',
 'pm',
 'portfolio',
 'property',
 'settings',
 'state'}

In [4]:
#List of columns in the given tables. Stored as a dictionary
meta_parse_json['table_column']

{'object': {'access_instructions',
  'category',
  'category_id',
  'created_at',
  'creator_id',
  'data',
  'description',
  'enter_permission',
  'estimate',
  'estimate_note',
  'flow_type',
  'future_schedule_date',
  'has_access_instructions',
  'is_emergency',
  'is_mandatory',
  'is_resident_responsible',
  'is_tenant_caused',
  'is_urgent',
  'is_vacant',
  'issue_id',
  'job_id',
  'job_rating',
  'location_id',
  'max_cost',
  'max_cost_hard',
  'name',
  'needs_help',
  'order_number',
  'owner_slug',
  'pm_id',
  'problem_id',
  'property_id',
  'pw_id',
  'rating_comments',
  'ref_job_id',
  'ref_service_request_id',
  'scheduled_end',
  'scheduled_start',
  'slug',
  'source',
  'state_id',
  'state_instance_id',
  'state_updated_at',
  'summary',
  'tenant_id',
  'tenant_slug',
  'updated_at',
  'use_scheduler',
  'vendor',
  'vendor_description',
  'vendor_id',
  'vendor_locked_in',
  'vendor_name',
  'vendor_pm_relation',
  'vendor_rating'},
 'state': {'description', 

In [5]:
# Datatypes of the given columns based on their values in each table
meta_parse_json['data_type_data']

{'object': {'job_id': int,
  'order_number': str,
  'name': str,
  'property_id': int,
  'tenant_id': int,
  'vendor_id': NoneType,
  'pm_id': int,
  'state_id': int,
  'use_scheduler': int,
  'description': str,
  'max_cost': int,
  'estimate': NoneType,
  'estimate_note': NoneType,
  'scheduled_start': NoneType,
  'created_at': str,
  'updated_at': str,
  'job_rating': NoneType,
  'vendor_rating': NoneType,
  'rating_comments': NoneType,
  'creator_id': int,
  'enter_permission': int,
  'scheduled_end': NoneType,
  'state_instance_id': str,
  'needs_help': int,
  'is_urgent': int,
  'vendor_description': str,
  'future_schedule_date': NoneType,
  'is_vacant': int,
  'access_instructions': NoneType,
  'has_access_instructions': int,
  'category_id': NoneType,
  'flow_type': str,
  'is_emergency': int,
  'pw_id': NoneType,
  'state_updated_at': str,
  'data': NoneType,
  'ref_job_id': NoneType,
  'source': str,
  'ref_service_request_id': NoneType,
  'slug': str,
  'is_mandatory': int,

In [6]:
# Values that goes in each column against each table
meta_parse_json['table_values']

{'object': {'job_id': 20,
  'order_number': '2-20',
  'name': 'dfafds',
  'property_id': 6,
  'tenant_id': 51,
  'vendor_id': None,
  'pm_id': 2,
  'state_id': 3,
  'use_scheduler': 1,
  'description': 'bing bong',
  'max_cost': 332,
  'estimate': None,
  'estimate_note': None,
  'scheduled_start': None,
  'created_at': '2020-09-04 23:37:39',
  'updated_at': '2020-09-05 02:02:07',
  'job_rating': None,
  'vendor_rating': None,
  'rating_comments': None,
  'creator_id': 1,
  'enter_permission': 0,
  'scheduled_end': None,
  'state_instance_id': '43ms2',
  'needs_help': 0,
  'is_urgent': 0,
  'vendor_description': 'fdgsgsffdadfggfsd fgsf dgsgf dsgdfs',
  'future_schedule_date': None,
  'is_vacant': 0,
  'access_instructions': None,
  'has_access_instructions': 0,
  'category_id': None,
  'flow_type': 'job',
  'is_emergency': 0,
  'pw_id': None,
  'state_updated_at': '2020-09-04 23:37:39',
  'data': None,
  'ref_job_id': None,
  'source': 'Internal Web Form',
  'ref_service_request_id': N

In [7]:
# Relationship between each tables. Note set() means its a root table
meta_parse_json['parent_child_tables']

{'object': set(),
 'state': {'object'},
 'property': {'object'},
 'pm': {'object', 'property'},
 'contacts': {'pm'},
 'company': {'object', 'pm'},
 'settings': {'company'},
 'payment_methods': {'company'},
 'portfolio': {'property'}}

In [8]:
# executing the various create and insert queries
create_load_automated_tables(DB, 'sample.json')


                         CREATE TABLE IF NOT EXISTS
                         "contacts"
                         ("is_primary" bigint,  "user_id" bigint,  "contact_id" bigint,  "method" str,  "value" str,  "is_valid" bigint,  "created_at" str,  "updated_at" str,  "is_textable" str,  "type" str,  "opted_out" bigint,  
                         );
                    

                    INSERT INTO 
                        "contacts"
                    ("is_primary","user_id","contact_id","method","value","is_valid","created_at","updated_at","is_textable","type","opted_out",)
                    VALUES
                    (1,2,3,email,pm@latchel.com,1,2020-09-04 13:43:34,2020-09-04 13:43:34,None,None,None,)
                 

                         CREATE TABLE IF NOT EXISTS
                         "pm"
                         ("user_id" bigint,  "user_type" str,  "name" str,  "created_at" str,  "updated_at" str,  "sms_enabled" bigint,  "email_enabled" bigint,  "critical_note" str